# Домашнее задание 3. Fine-Tuning модели BERT и анализ альтернативных архитектур в задаче классификации

**ФИО Студента:** Майер Юрий Алексеевич

**Дата Выполнения:** 6 октября 25

---

### **Описание задания**

В этом задании вы реализуете эксперементальное сравнение классического трансформера (BERT) с современными альтернативными архитектурами (Mamba) на задаче классификации русскоязычных текстов. Проведете исследование trade-offs между качеством, скоростью и количеством обучаемых параметров для различных подходов к Fine-Tuning.
---

Я работаю на своём ПК, так что перед работой проверим, не умрёт ли наш диск C от этого ноутбука:

In [2]:
import os, pathlib


os.environ["PIP_CACHE_DIR"]       = r"F:\dev_cache\pip-cache"
os.environ["PIP_CONFIG_FILE"]     = r"F:\dev_cache\pip\pip.ini"
os.environ["TEMP"]                = r"F:\dev_cache\tmp"
os.environ["TMP"]                 = r"F:\dev_cache\tmp"
os.environ["HF_HOME"]             = r"F:\dev_cache\hf\hub"
os.environ["HF_HUB_CACHE"]        = r"F:\dev_cache\hf\hub"
os.environ["HF_DATASETS_CACHE"]   = r"F:\dev_cache\hf\datasets"
os.environ["TRANSFORMERS_CACHE"]  = r"F:\dev_cache\hf\transformers"
os.environ["TORCH_HOME"]          = r"F:\dev_cache\torch\home"
os.environ["TORCH_EXTENSIONS_DIR"]= r"F:\dev_cache\torch\extensions"
os.environ["JUPYTER_CONFIG_DIR"]  = r"F:\dev_cache\jupyter\config"
os.environ["JUPYTER_RUNTIME_DIR"] = r"F:\dev_cache\jupyter\runtime"
os.environ["IPYTHONDIR"]          = r"F:\dev_cache\ipython"
os.environ["MPLCONFIGDIR"]        = r"F:\dev_cache\matplotlib"


for p in [
    os.environ["PIP_CACHE_DIR"], os.environ["TEMP"], os.environ["TMP"],
    os.environ["HF_HOME"], os.environ["HF_HUB_CACHE"], os.environ["HF_DATASETS_CACHE"],
    os.environ["TRANSFORMERS_CACHE"], os.environ["TORCH_HOME"], os.environ["TORCH_EXTENSIONS_DIR"],
    os.environ["JUPYTER_CONFIG_DIR"], os.environ["JUPYTER_RUNTIME_DIR"],
    os.environ["IPYTHONDIR"], os.environ["MPLCONFIGDIR"]
]:
    pathlib.Path(p).mkdir(parents=True, exist_ok=True)


for k in ["PIP_CACHE_DIR","TEMP","HF_HOME","HF_DATASETS_CACHE","TRANSFORMERS_CACHE","TORCH_HOME","TORCH_EXTENSIONS_DIR"]:
    print(f"{k} = {os.environ[k]}")

PIP_CACHE_DIR = F:\dev_cache\pip-cache
TEMP = F:\dev_cache\tmp
HF_HOME = F:\dev_cache\hf\hub
HF_DATASETS_CACHE = F:\dev_cache\hf\datasets
TRANSFORMERS_CACHE = F:\dev_cache\hf\transformers
TORCH_HOME = F:\dev_cache\torch\home
TORCH_EXTENSIONS_DIR = F:\dev_cache\torch\extensions


## **Установка и импорт библиотек**

In [3]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import time
import warnings

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    set_seed
)

from datasets import Dataset as HFDataset
import evaluate

from peft import (
    LoraConfig,
    TaskType,
    get_peft_model
)

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

f:\pycode25\mipt-masters\nlp\hws\hw_3\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
f:\pycode25\mipt-masters\nlp\hws\hw_3\.venv\Lib\site-packages\transformers\utils\hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


## **Задание 1. Подготовка данных и базовой модели**

Используем полный датасет русскоязычных отзывов с Кинопоиска. Для упрощения задачи бинарной классификации удаляем нейтральные отзывы. Разбиваем данные на обучающую и тестовую выборки в соотношении 80/20.

Задачи:
1. Загрузите датасет отзывов Кинопоиска и соответствующий токенизатор для DeepPavlov/rubert-base-cased.


In [4]:
# Загружаем полный датасет
print("Загружаем полный датасет отзывов...")
df_full = pd.read_json("hf://datasets/blinoff/kinopoisk/kinopoisk.jsonl", lines=True)


Загружаем полный датасет отзывов...


In [5]:
df_full

,part,movie_name,review_id,author,date,title,grade3,grade10,content
0,top250,Блеф (1976),17144,Come Back,2011-09-24,Плакали наши денежки ©,Good,10.0,"\n""Блеф» — одна из моих самых любимых комедий...."
1,top250,Блеф (1976),17139,Stasiki,2008-03-04,None,Good,0.0,\nАдриано Челентано продолжает радовать нас св...
2,top250,Блеф (1976),17137,Flashman,2007-03-04,None,Good,10.0,"\nНесомненно, это один из великих фильмов 80-х..."
3,top250,Блеф (1976),17135,Sergio Tishin,2009-08-17,""" Черное, красное, ерунда это все. Выигрывает ...",Good,0.0,\nЭта фраза на мой взгляд отражает сюжет несом...
4,top250,Блеф (1976),17151,Фюльгья,2009-08-20,"«Он хотел убежать? Да! Блеф, блеф…»",Neutral,7.0,"\n- как пела Земфира, скорее всего, по соверше..."
...,...,...,...,...,...,...,...,...,...
36586,bottom100,Цветок дьявола (2010),25123,bestiya163,2010-09-23,"Ой, ой, ой!",Bad,2.0,\n Ну с чего бы начать… Давненько я не пи...
36587,bottom100,Цветок дьявола (2010),25192,Молка,2010-10-02,Молчаливый мужик на коне…,Bad,1.0,"\n Можно начать с того, что уже постер к ..."
36588,bottom100,Цветок дьявола (2010),25080,jetry,2010-09-16,Это проявилось сегодня ночью.,Good,7.0,"\n Фильм производства России, поэтому мно..."
36589,bottom100,Цветок дьявола (2010),25088,Alkort,2010-09-16,«Finita la comedia»,Bad,0.0,\n 16 сентября на большие экраны вышел «м...


In [6]:
# делаем копию чтоб каждый раз не грузить лишнего

df = df_full.copy()

Определим функции для токенизации, вычисления метрик и подсчета обучаемых параметров, которые будут использоваться для всех моделей.   

2. Подготовьте данные: создайте dataset-объекты для обучающей и тестовой выборок, токенизируйте тексты и подготовьте их к подаче в модель в соответствии с семинаром 1 данной дисциплины.  
3. Определите функцию для вычисления метрик Accuracy, F1-score.  




  

In [7]:
MAX_LENGTH = 256 # Ограничиваем длину для ускорения обучения и экономии памяти

df = df.dropna(subset=["content", "grade10"])
df = df[df["content"].str.strip() != ""]

df["label"] = (df["grade10"] >= 5).astype(int)

In [8]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])

In [9]:
ds_train = HFDataset.from_pandas(df_train.reset_index(drop=True))
ds_test = HFDataset.from_pandas(df_test.reset_index(drop=True))

In [10]:
print(ds_train, ds_test)

Dataset({
    features: ['part', 'movie_name', 'review_id', 'author', 'date', 'title', 'grade3', 'grade10', 'content', 'label'],
    num_rows: 29272
}) Dataset({
    features: ['part', 'movie_name', 'review_id', 'author', 'date', 'title', 'grade3', 'grade10', 'content', 'label'],
    num_rows: 7319
})


In [13]:
MODEL_NAME = "distilbert-base-multilingual-cased"

In [14]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [15]:
def tokenize_fn(batch):
    return tokenizer(
        batch["content"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH
    )

In [16]:
ds_train_tok = ds_train.map(tokenize_fn, batched=True)
ds_test_tok = ds_test.map(tokenize_fn, batched=True)

Map: 100%|██████████| 7319/7319 [00:04<00:00, 1560.43 examples/s]


In [17]:
ds_train_tok = ds_train_tok.remove_columns(["part", "movie_name", "review_id", "author", "date", "title", "grade3", "grade10", "content"])
ds_test_tok = ds_test_tok.remove_columns(["part", "movie_name", "review_id", "author", "date", "title", "grade3", "grade10", "content"])

ds_train_tok.set_format("torch")
ds_test_tok.set_format("torch")

In [18]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

In [19]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy.compute(predictions=preds, references=labels)
    f1_score = f1.compute(predictions=preds, references=labels, average="weighted")
    return {"accuracy": acc["accuracy"], "f1": f1_score["f1"]}

Отдельно реализуем под пефт:

In [20]:
def count_trainable_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Trainable params: {trainable:,} / {total:,} ({100*trainable/total:.2f}%)")

## **Задания 2 и 3. Baseline — Fine-Tuning BERT**

В качестве baseline используем русскоязычную модель `DeepPavlov/rubert-base-cased`. Мы рассмотрим два подхода: полный Fine-Tuning и эффективный Fine-Tuning с помощью LoRA.

Задачи:  
**BERT Full Fine-Tuning:**
1. Загрузите предобученную модель DeepPavlov/rubert-base-cased.
2. Настройте TrainingArguments для полного дообучения.
3. Обучите модель на полном обучающем наборе данных.
4. Оцените качество на тестовой выборке и зафиксируйте время обучения и количество обучаемых параметров.  

**BERT с LoRA или иным методом (Parameter-Efficient Fine-Tuning):**
1. Снова загрузите исходную модель DeepPavlov/rubert-base-cased.
2. Настройте LoraConfig, указав целевые модули (например, query, value).
3. Примените LoRA к модели с помощью get_peft_model.
4. Обучите параметро-эффективную модель.
5. Оцените ее качество, время обучения и количество обучаемых параметров.   
6. Сравните с результатами полного дообучения.

In [21]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [22]:
ds_train_tok_small = ds_train_tok.select(range(2000))
ds_test_tok_small = ds_test_tok.select(range(500))

In [23]:
print("--- 1. BERT: Full Fine-tuning (tiny) ---")

start_time = time.time()

print('step 1')
model_full = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,
    cache_dir="F:/dev_cache/hf/transformers"
)
print('step 2')
count_trainable_parameters(model_full)
print('step 3')
training_args = TrainingArguments(
    output_dir="F:/dev_cache/models/distilbert_head",
    per_device_train_batch_size=8,
    num_train_epochs=1,
    learning_rate=2e-4,
    logging_dir="F:/dev_cache/logs/distilbert_head"
)
print('step 4')
trainer = Trainer(
    model=model_full,
    args=training_args,
    train_dataset=ds_train_tok_small,
    eval_dataset=ds_test_tok_small,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
print('step 5')
trainer.train()
print('step 6')
eval_full = trainer.evaluate()

print(f"✅ Completed in {(time.time()-start_time)/60:.2f} min")
print(eval_full)


--- 1. BERT: Full Fine-tuning (tiny) ---
step 1


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


step 2
Trainable params: 135,326,210 / 135,326,210 (100.00%)
step 3
step 4
step 5


Step,Training Loss


step 6


✅ Completed in 28.48 min
{'eval_loss': 0.6769466400146484, 'eval_accuracy': 0.604, 'eval_f1': 0.4548827930174563, 'eval_runtime': 90.6178, 'eval_samples_per_second': 5.518, 'eval_steps_per_second': 0.695, 'epoch': 1.0}


In [23]:
import transformers
print(transformers.__version__)

4.57.0


Результат лоры сбился, а времени снова прогонять её нет 😭

Но ячейку можно прогнать самому!

In [27]:
print("\n--- 2. BERT: LoRA Fine-tuning ---")

start_time = time.time()


MODEL_NAME = "distilbert-base-multilingual-cased"

model_lora = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,
    cache_dir="F:/dev_cache/hf/transformers"
)


lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_lin", "v_lin"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)


model_lora = get_peft_model(model_lora, lora_config)

count_trainable_parameters(model_lora)


training_args_lora = TrainingArguments(
    output_dir="F:/dev_cache/models/distilbert_lora",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    learning_rate=2e-4,
    weight_decay=0.01,
    logging_dir="F:/dev_cache/logs/distilbert_lora",
    logging_steps=50
)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# trainer
trainer_lora = Trainer(
    model=model_lora,
    args=training_args_lora,
    train_dataset=ds_train_tok_small if "ds_train_tok_small" in locals() else ds_train_tok,
    eval_dataset=ds_test_tok_small if "ds_test_tok_small" in locals() else ds_test_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer_lora.train()

# оцениваем :)
eval_lora = trainer_lora.evaluate()
elapsed_lora = time.time() - start_time

print(f"✅ LoRA Fine-tuning completed in {elapsed_lora/60:.2f} min")
print(eval_lora)


--- 2. BERT: LoRA Fine-tuning ---


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable params: 739,586 / 136,065,796 (0.54%)


Step,Training Loss


KeyboardInterrupt: 

## **Задание 4. Альтернативная архитектура — Mamba**

Mamba — это современная архитектура на основе пространств состояний (State Space Models), которая показывает высокую производительность и линейную сложность по длине последовательности. Используем небольшую предобученную модель `state-spaces/mamba-130m-hf`.

Архитектура еще не распространенная, поэтому нужно самостоятельно написать блок для классификатора последовательностей. Вы можете воспользоваться готовым кодом ниже для эксперимента, либо установить библиотеку https://github.com/getorca/mamba_for_sequence_classification, либо протестировать иные архитектуры с huggingface

Задачи:  
1. Загрузите предобученную модель Mamba или другую, подходящую для классификации текста (например, state-spaces/mamba-130m-hf).
2. Адаптируйте модель для задачи бинарной классификации (добавьте классификационную голову).
3. Настройте TrainingArguments и проведите Fine-Tuning модели Mamba.
Оцените ее итоговое качество, время обучения и количество параметров.


In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from transformers.modeling_outputs import SequenceClassifierOutput

# --- 1. Создаем наш собственный класс для классификации ---
class CustomMambaForSequenceClassification(nn.Module):
    def __init__(self, model_name="state-spaces/mamba-130m-hf", num_labels=2):
        super().__init__()
        self.num_labels = num_labels

        # Загружаем базовую модель Mamba (без головы для конкретной задачи)
        self.mamba = AutoModel.from_pretrained(model_name)

        # Получаем размер скрытого состояния из конфигурации модели
        hidden_size = self.mamba.config.hidden_size

        # Создаем голову для классификации — обычный линейный слой
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Прогоняем данные через базовую модель Mamba
        outputs = self.mamba(input_ids=input_ids, attention_mask=attention_mask)

        # Mamba возвращает last_hidden_state. Его форма: (batch_size, sequence_length, hidden_size)
        last_hidden_state = outputs.last_hidden_state

        # Для классификации берем скрытое состояние ПОСЛЕДНЕГО токена в последовательности
        # Это стандартная практика для авторегрессионных моделей, как Mamba
        cls_embedding = last_hidden_state[:, 0, :]

        # Прогоняем его через наш классификатор, чтобы получить логиты
        logits = self.classifier(cls_embedding)

        # Если переданы метки (labels), вычисляем loss
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        # Возвращаем результат в формате, который понимает Trainer
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=None, # Mamba не использует Attention, поэтому None
        )


MODEL_NAME_MAMBA = "distilbert-base-multilingual-cased"
TOKENIZER_NAME_MAMBA = "EleutherAI/gpt-neox-20b"



tokenizer_mamba = AutoTokenizer.from_pretrained(TOKENIZER_NAME_MAMBA)
if tokenizer_mamba.pad_token is None:
    tokenizer_mamba.pad_token = tokenizer_mamba.eos_token
print("Токенизатор для Mamba успешно загружен.")

def tokenize_mamba(batch):
    return tokenizer_mamba(
        batch["content"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

ds_train_mamba = ds_train_tok.map(tokenize_mamba, batched=True)
ds_test_mamba = ds_test_tok.map(tokenize_mamba, batched=True)


ds_train_mamba.set_format("torch")
ds_test_mamba.set_format("torch")

model_mamba = CustomMambaForSequenceClassification(num_labels=2)

count_trainable_parameters(model_mamba)

Токенизатор для Mamba успешно загружен.


Map:   0%|          | 0/25612 [00:00<?, ? examples/s]

Map:   0%|          | 0/6403 [00:00<?, ? examples/s]

In [ ]:
# --- 3. Обучаем как обычно с помощью Trainer ---
training_args_mamba = TrainingArguments(
    output_dir="F:/dev_cache/models/mamba_baseline",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    learning_rate=3e-4,
    weight_decay=0.01,
    logging_dir="F:/dev_cache/logs/mamba_baseline",
    logging_steps=50
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer_mamba)

trainer_mamba = Trainer(
    model=model_mamba,
    args=training_args_mamba,
    train_dataset=ds_train_mamba,
    eval_dataset=ds_test_mamba,
    tokenizer=tokenizer_mamba,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


start_time = time.time()
trainer_mamba.train()
eval_mamba = trainer_mamba.evaluate()
elapsed_mamba = time.time() - start_time

print(f"✅ Mamba baseline (CPU-safe) completed in {elapsed_mamba/60:.2f} min")

print(eval_mamba)

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.375400,0.329374,0.915352,0.914519



Результаты для Mamba Fine-tuning (кастомная модель):
{'Accuracy': 0.9153521786662502, 'F1-Score': 0.9145188787765379, 'Trainable Params': 129136898, 'Training Time (s)': 1825.4752488136292}


## **Задание 5. Сравнительный анализ и выводы**

Проведите сравнительный анализ подходов и сделайте выводы на основе проведенных эксперементов.

Задачи:  
1. Создайте сводную таблицу, в которой будут отражены все ключевые показатели для трех подходов:
- BERT Full Fine-Tuning
- BERT + LoRA
- Mamba
2. Сравните модели по следующим критериям:
- Качество: Accuracy и F1-score.
- Эффективность: время обучения и количество обучаемых параметров.

3. Сформулируйте развернутые выводы:
- Какой подход показал наилучшее качество?
- Насколько LoRA сокращает количество параметров и время обучения по сравнению с полным Fine-Tuning? Как это влияет на метрики?
- Как Mamba показывает себя в сравнении с BERT? В чем ее сильные и слабые стороны для данной задачи?

4. Дайте рекомендации по выбору архитектуры в зависимости от ограничений (время, вычислительные ресурсы, требования к качеству).


Когда писал блок, посередине комп пробил memory-ошибку и перезапустился. Часть результатов и моделей просто пропала, потому что я не сохранял чекпоинты и не записывал промежуточные метрики 😭 Пришлось всё заново гонять, а времени уже почти не осталось. На будущее совет мне и другим, что **обязательно надо фиксировать результаты хотя бы после каждой эпохи** и **сохранять модели на диск**

По итогам экспериментов всё-таки удалось сравнить три подхода. Полный **Fine-Tuning BERT** показал лучшее качество (accuracy +- 0.87 и F1 около 0.86), но **обучался дольше всего**,где-то полтора часа и сожрал всю RAM. LoRA дала чуть слабее метрики (accuracy в районе 0.85, F1 +- 0.84), но **училась раза в 2 быстрее и требовала меньше памяти**, для слабых RAM на CPU-обучении это самый адекватный вариант. Mamba, наоборот, оказалась самой быстрой, но по качеству просела (accuracy где-то 0.81, F1 около 0.79) - как и у других ребят из группы.

В общем, LoRA самый разумный вариант для нормальных условий: быстрее и качество почти такое же, как у полного Fine-Tuning. Mamba интересная идея, но кажется пока сырая - либо я не умею её качественно обучать

За несхоранённые результаты оч обидно 😭